<a href="https://colab.research.google.com/github/sid0nair/3D-CNN-/blob/main/wala_implement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%pip install \
    open3d \
    h5py \
    pymcubes \
    pytorch_wavelets \
    PyWavelets \
    spconv-cu120 \
    omegaconf \
    fasteners \
    trimesh \
    matplotlib \
    plotly

In [8]:
import matplotlib.pyplot as plt, matplotlib.image as mpimg
import trimesh, plotly.graph_objs as go, numpy as np, json, glob


def display_images(image_files):
    fig, axes = plt.subplots(1, len(image_files), figsize=(5*len(image_files),5))
    if len(image_files)==1: axes=[axes]
    for ax, f in zip(axes, image_files):
        ax.imshow(mpimg.imread(f)); ax.axis('off')
    plt.show()

def display_mesh(mesh_file):
    m = trimesh.load(mesh_file)
    v, f = m.vertices, m.faces
    fig = go.Figure([go.Mesh3d(
        x=v[:,0], y=v[:,1], z=v[:,2],
        i=f[:,0], j=f[:,1], k=f[:,2],
        lighting=dict(ambient=0.5, diffuse=1, specular=0.5, roughness=0.5)
    )])
    fig.update_layout(scene=dict(xaxis_visible=False,yaxis_visible=False,zaxis_visible=False))
    fig.show()

def display_voxel(json_file):
    data = json.load(open(json_file))
    occ = np.array(data["occupancy"])
    res = data["resolution"]
    vox = np.zeros((res+1,)*3)
    vox[occ[:,0],occ[:,1],occ[:,2]] = 1
    fig = plt.figure(); ax = fig.add_subplot(projection='3d')
    ax.voxels(vox, facecolors="#B4B7C2", edgecolor="k"); plt.axis('off'); plt.show()

In [9]:
# install condacolab (this will reboot your runtime)
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [10]:
!git clone https://github.com/AutodeskAILab/WaLa.git
%cd WaLa

Cloning into 'WaLa'...
remote: Enumerating objects: 386, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 386 (delta 118), reused 104 (delta 47), pack-reused 167 (from 3)
Receiving objects: 100% (386/386), 11.10 MiB | 16.96 MiB/s, done.
Resolving deltas: 100% (137/137), done.
/content/WaLa/WaLa


In [11]:
# PyTorch and Lightning
!mamba install -y -c pytorch pytorch=2.1.2 torchvision=0.16.2 pytorch-lightning=2.1.0

# 3D/mesh libraries and addons
!mamba install -y -c conda-forge open3d h5py pymcubes fasteners ftfy regex pytorch-wavelets pywavelets spconv-cu120 einops timm trimesh scikit-image seaborn

# diffusers, transformers, huggingface hub, open-clip
!mamba install -y -c conda-forge diffusers transformers huggingface_hub open-clip-torch=2.24.0

# others via pip
!pip install cloudpathlib backoff xformers Pillow plotly


Looking for: ['pytorch=2.1.2', 'torchvision=0.16.2', 'pytorch-lightning=2.1.0']

[+] 0.0s
pytorch/linux-64      ⣾  
pytorch/noarch        ⣾  
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  pytorch/noarch                                                No change
conda-forge/linux-64                                          No change
pytorch/linux-64                                              No change
[+] 0.1s
conda-forge/noarch   1%[+] 0.2s
conda-forge/noarch  49%[+] 0.3s
conda-forge/noarch  98%conda-forge/noarch                                

Pinned packages:
  - python 3.11.*
  - python 3.11.*
  - python_abi 3.11.* *cp311*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  All requested packages already installed


Looking for: ['open3d', 'h5py', 'pymcubes', 'fasteners', 'ftfy', 'regex', 'pytorch-wavelets', 'pywavelets', 'spconv-cu120', 'einops', 'timm', 'trimesh', 'scikit-image', 'seaborn']

conda-forge/linux-64                                        Using cache
co

In [12]:
from google.colab import files
# this pops up a file-picker; choose motor_png.zip
uploaded = files.upload()
# unzip into /content/data (you can pick any folder)
!unzip -q motor_png.zip -d /content/data

Saving motor_png.zip to motor_png.zip
replace /content/data/motor png/163406588-34-5tk20a-aw2u/5tk20a-aw2u.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/motor png/163406588-34-5tk20a-aw2u/5tk20a-aw2u_640x480_back.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/motor png/163406588-34-5tk20a-aw2u/5tk20a-aw2u_640x480_bottom.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/motor png/163406588-34-5tk20a-aw2u/5tk20a-aw2u_640x480_front.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/motor png/163406588-34-5tk20a-aw2u/5tk20a-aw2u_640x480_iso.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/motor png/163406588-34-5tk20a-aw2u/5tk20a-aw2u_640x480_left.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/motor png/163406588-34-5tk20a-aw2u/5tk20a-aw2u_640x480_right.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/motor png/163406588-34-5tk20a-aw2u/5tk20a-aw2u_640x480_top.pn

In [14]:
%%bash
INPUT_ROOT="/content/data/motor png"
OUTPUT_ROOT="/content/wala_out"
mkdir -p "$OUTPUT_ROOT"

# find all motor subfolders
motors=( "$INPUT_ROOT"/* )
total=${#motors[@]}
echo "Found $total motors to process."

i=0
for motor_dir in "${motors[@]}"; do
  ((i++))
  motor=$(basename "$motor_dir")
  out="$OUTPUT_ROOT/$motor"
  mkdir -p "$out"

  echo "[$i/$total] → Processing motor '$motor'…"

  # make temp dir for ordered dm6 symlinks
  TMP=$(mktemp -d)
  ln -s "$motor_dir"/*_front.png  "$TMP/0.png"
  ln -s "$motor_dir"/*_right.png  "$TMP/1.png"
  ln -s "$motor_dir"/*_back.png   "$TMP/2.png"
  ln -s "$motor_dir"/*_left.png   "$TMP/3.png"
  ln -s "$motor_dir"/*_top.png    "$TMP/4.png"
  ln -s "$motor_dir"/*_bottom.png "$TMP/5.png"

  # run WaLa 6-view on the numeric symlinks
  python run.py \
    --model_name ADSKAILab/WaLa-DM6-1B \
    --dm6 \
      "$TMP/0.png" \
      "$TMP/1.png" \
      "$TMP/2.png" \
      "$TMP/3.png" \
      "$TMP/4.png" \
      "$TMP/5.png" \
    --output_dir "$out" \
    --output_format obj \
    --scale 1.5 \
    --diffusion_rescale_timestep 10

  status=$?
  # cleanup
  rm -rf "$TMP"

  if [ $status -eq 0 ]; then
    echo "[$i/$total] ✓ Done with $motor"
  else
    echo "[$i/$total] ✗ Error on $motor" >&2
  fi
  echo "--------------------------------"
done

echo "All $total motors processed."

Found 7 motors to process.
[1/7] → Processing motor '163406588-34-5tk20a-aw2u'…
Loading model
'DotDict' object has no attribute 'dataset_path'
'DotDict' object has no attribute 'low_avg'
'DotDict' object has no attribute 'low_avg'
Low avg used : None high value: 63
cond_emb_dim: 1024
Input resolution: 224
Vocab size: N/A
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'
tensor([[0, 1, 2, 3, 4, 5]], device='cuda:0')
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'
[1/7] ✓ Done with 163406588-34-5tk20a-aw2u
--------------------------------
[2/7] → Processing motor '163406610-34-2tk3a-aw2u'…
Loading model
'DotDict' object has no attribute 'dataset_path'
'DotDict' object has no attribute 'low_avg'


/usr/local/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/usr/local/lib/python3.11/site-packages/spconv/pytorch/functional.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  _TORCH_CUSTOM_FWD = amp.custom_fwd(cast_inputs=torch.float16)
/usr/local/lib/python3.11/site-packages/spconv/pytorch/functional.py:96: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @_TORCH_CUSTOM_BWD

In [23]:
import plotly.io as pio
pio.renderers.default = "colab"

import trimesh
import plotly.graph_objs as go

def display_mesh(mesh_file):
    # load, merge if Scene
    loaded = trimesh.load(mesh_file)
    if isinstance(loaded, trimesh.Scene):
        mesh = trimesh.util.concatenate(loaded.dump())
    else:
        mesh = loaded

    v, f = mesh.vertices, mesh.faces
    fig = go.Figure([go.Mesh3d(
        x=v[:,0], y=v[:,1], z=v[:,2],
        i=f[:,0], j=f[:,1], k=f[:,2],
        color='#B4B7C2',
        lighting=dict(ambient=0.5, diffuse=1, specular=0.5, roughness=0.5),
        hoverinfo='none'
    )])
    fig.update_layout(
        scene=dict(
            xaxis_visible=False,
            yaxis_visible=False,
            zaxis_visible=False,
            aspectmode='data',
            # you can tweak the camera here if it looks flat:
            camera=dict(eye=dict(x=2, y=2, z=2))
        ),
        margin=dict(t=0, r=0, l=0, b=0)
    )
    fig.show()

In [24]:
from glob import glob

all_objs = glob('/content/wala_out/*/*/*.obj')
print("Meshes:", all_objs)

# show the first mesh
display_mesh(all_objs[0])

Meshes: ['/content/wala_out/163406588-34-5tk20a-aw2u/0/798d32c8-236e-425f-9bc5-0c88917ae676.obj', '/content/wala_out/163406634-34-3rk15a-ua/0/04db3480-0036-4917-80bb-06df3cd9fbff.obj', '/content/wala_out/163406614-34-2tk3a-cw2e/0/572929a0-9d1f-46b9-bdae-4a7aaf968ef4.obj', '/content/wala_out/163406617-34-3tk6a-aw2u/0/3fb8a2c5-abc8-4d0b-94b2-75fab3be538c.obj', '/content/wala_out/163406620-34-3tk6a-cw2e/0/54359fc5-2726-45b5-8497-e15d5316e1b3.obj', '/content/wala_out/163406610-34-2tk3a-aw2u/0/c9eac2e7-c713-46ad-b637-a40bdd373525.obj', '/content/wala_out/163406648-34-3rk15a-uam/0/7e498624-abeb-4238-90d9-3544199ca385.obj']


In [17]:
import glob
all_objs = glob.glob('/content/wala_out/*/*/*.obj')
print(f"Found {len(all_objs)} mesh files:")
for p in all_objs:
    print("  ", p)

Found 7 mesh files:
   /content/wala_out/163406588-34-5tk20a-aw2u/0/798d32c8-236e-425f-9bc5-0c88917ae676.obj
   /content/wala_out/163406634-34-3rk15a-ua/0/04db3480-0036-4917-80bb-06df3cd9fbff.obj
   /content/wala_out/163406614-34-2tk3a-cw2e/0/572929a0-9d1f-46b9-bdae-4a7aaf968ef4.obj
   /content/wala_out/163406617-34-3tk6a-aw2u/0/3fb8a2c5-abc8-4d0b-94b2-75fab3be538c.obj
   /content/wala_out/163406620-34-3tk6a-cw2e/0/54359fc5-2726-45b5-8497-e15d5316e1b3.obj
   /content/wala_out/163406610-34-2tk3a-aw2u/0/c9eac2e7-c713-46ad-b637-a40bdd373525.obj
   /content/wala_out/163406648-34-3rk15a-uam/0/7e498624-abeb-4238-90d9-3544199ca385.obj


In [21]:
display_mesh(all_objs[0])